## Prepare the data for the model ##

The processed data used in the paper is available on the github repo. However, detailed information on steps required to process the data is not. These pre-processing notebooks aim to create a dataset that can be easily used by the model.

We take the following steps:

<img src="../images/data_path.png" alt="Data Path" style="width:60%;">

Demean, detrend, and resample are done in the [Data_Cleaning](Data_Cleaning.ipynb) notebook.



Read each file and segment them into one hour windows. Each file will contain 28,800 samples per hour at an 8 Hz sampling rate.

In [30]:
import os
from obspy import read
import glob
import numpy as np
from obspy import UTCDateTime

# Define the processed and output folder paths
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define the target segment duration (1 hour) in seconds
segment_duration = 3600  # seconds

# Process each processed file for segmentation
for file_path in glob.glob(f"{processed_folder}/*.mseed"):
    # Read the processed file
    st = read(file_path)
    
    # Normalize the traces in the stream object - done in data_cleaning
    #st.normalize()
    
    # Split each trace in the Stream object into one-hour segments
    for tr in st:
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time
        
        # Loop over each one-hour segment
        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            # Format the filename for the segment
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.mseed"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)
            
            # Save the one-hour segment as a new file
            try:
                segment.write(segment_filepath, format="MSEED")
                print(f"Saved segment file: {segment_filename}")
            except Exception as e:
                print(f"Error writing segment {segment_filename}: {e}")

            # Move to the next hour
            segment_start = segment_end

            # # Save the one-hour segment as a new file
            # segment.write(segment_filepath, format="MSEED")
            # print(f'Saved segment file: {segment_filepath}')

            # # Move to the next hour
            # segment_start = segment_end

Saved segment file: 9F_NUPH_HHE_20210401T224800.mseed
Saved segment file: 9F_NUPH_HHE_20210401T234800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T004800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T014800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T024800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T034800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T044800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T054800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T064800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T074800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T084800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T094800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T104800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T114800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T124800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T134800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T144800.mseed
Saved segment file: 9F_NUPH_HHE_20210402T154800.mseed
Saved segment file: 9F_NUPH_

In [ ]:
# check all files in the segmented folder for nan data

for file_path in glob.glob(f"{segmented_output_folder}/*.mseed"):
    st = read(file_path)
    for tr in st:
        if np.isnan(tr.data).any():
            print(f"File {file_path} contains NaN data")
        else:
            print(f"File {file_path} contains no NaN data")


done


Apply the Fourier Transform with the `specified window_length` and `hop_length parameters` to produce a spectrogram of size (96, 128) for each one-hour segment. The data is saved in `data\segmented`.

In [ ]:
import os
import glob
import numpy as np
from obspy import read
import librosa
from scipy.signal import resample
import time
from IPython.display import display, clear_output
import torch

# Define directories
input_folder = os.getcwd() + '/data/raw'
processed_folder = os.getcwd() + '/data/processed'
segmented_output_folder = os.getcwd() + '/data/segmented'
os.makedirs(segmented_output_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Define constants
target_sampling_rate = 8  # Hz
segment_duration = 3600  # 1 hour in seconds
window_length = 256
hop_length = 224
target_shape = (96, 128)

# Initialize an empty list to store spectrograms
spectrogram_list = []

# Step 1: Process each file in the input folder
for file_path in glob.glob(f"{input_folder}/*.mseed"):
    # Read the file
    st = read(file_path)
    
    # Process each trace
    for tr in st:
        # Resample the data to 8 Hz (assuming original sampling rate was 100 Hz)
        # this is done in data cleaning
        #tr.resample(target_sampling_rate)
        
        # Split into one-hour segments (28,800 samples each)
        start_time = tr.stats.starttime
        end_time = tr.stats.endtime
        segment_start = start_time

        while segment_start + segment_duration <= end_time:
            # Define the end time for the current segment
            segment_end = segment_start + segment_duration

            # Slice the trace to create a one-hour segment
            segment = tr.slice(starttime=segment_start, endtime=segment_end)
            
            ## NEW ##
            # Convert to PyTorch tensor
            segment_tensor = torch.tensor(segment, dtype=torch.float32)
            
            # Ensure correct dimensions (STFT expects 1D tensor or batch x time)
            if len(segment_tensor.shape) == 1:
                segment_tensor = segment_tensor.unsqueeze(0)
                
            # Compute the STFT
            stft_result = torch.stft(
                segment_tensor,
                n_fft=window_length,
                hop_length=hop_length,
                win_length=window_length,
                return_complex=True,
            )
                
            # Compute the spectrogram (magnitude squared)
            spectrogram = torch.abs(stft_result) ** 2

            # Convert to NumPy for further processing
            spectrogram = spectrogram.numpy()

            # Resize to a consistent shape if needed (example using interpolation)
            target_shape = (96, 128)  # Desired shape
            spectrogram_resized = np.resize(spectrogram, target_shape)

            # Log-transform the spectrogram
            spectrogram_log = np.log1p(spectrogram_resized)

            # Normalize between -1 and 1
            spectrogram_log = 2 * (spectrogram_log - np.min(spectrogram_log)) / np.ptp(spectrogram_log) - 1
            
            ## END NEW ##

            # Save the spectrogram (you could save the result as needed)
            segment_filename = f"{tr.stats.network}_{tr.stats.station}_{tr.stats.channel}_{segment_start.strftime('%Y%m%dT%H%M%S')}.npy"
            segment_filepath = os.path.join(segmented_output_folder, segment_filename)

            np.save(segment_filepath, spectrogram_log)

            # Append the spectrogram to the list
            spectrogram_list.append(spectrogram_log)
            
            # check the file for data outside the bounds of -1 and 1
            if np.any(spectrogram_log > 1) or np.any(spectrogram_log < -1):
                print(f"File {segment_filename} contains data outside the bounds of -1 and 1")

            # Move to the next hour
            segment_start = segment_end
            
            # display file being processed
            # time.sleep(0.1)
            # clear_output(wait=True)
            print(f'Processing file: {segment_filename}')
            
            #check file for nan data
            if np.isnan(spectrogram_log).any():
                print(f"File {segment_filename} contains NaN data")             

# Stack all spectrograms into a single numpy array and add batch and channel dimensions
all_spectrograms = np.stack(spectrogram_list)
all_spectrograms = all_spectrograms[:, np.newaxis, :, :]  # Shape: (batch_size, 1, 96, 128)
print(f"Shape of combined spectrogram array: {all_spectrograms.shape}")

# remove the single feature dimension (1)
all_spectrograms = np.squeeze(all_spectrograms)

# NOTE: FILE NAME IS HARD CODED HERE
# Save the combined spectrogram array to Input.npy
final_output_folder = os.getcwd()
input_filepath = os.path.join(final_output_folder, "Input.npy")
#input_filepath = os.path.join(final_output_folder + 'NUPH_analysis', "Input_nuph.npy")
np.save(input_filepath, all_spectrograms)

print(f"Saved combined spectrogram file: {input_filepath.split('/')[-1]}")


Processing file: 9F_NUPH_HHE_20210401T224800.npy
Processing file: 9F_NUPH_HHE_20210401T234800.npy
Processing file: 9F_NUPH_HHE_20210402T004800.npy
Processing file: 9F_NUPH_HHE_20210402T014800.npy
Processing file: 9F_NUPH_HHE_20210402T024800.npy
Processing file: 9F_NUPH_HHE_20210402T034800.npy
Processing file: 9F_NUPH_HHE_20210402T044800.npy
Processing file: 9F_NUPH_HHE_20210402T054800.npy
Processing file: 9F_NUPH_HHE_20210402T064800.npy
Processing file: 9F_NUPH_HHE_20210402T074800.npy
Processing file: 9F_NUPH_HHE_20210402T084800.npy
Processing file: 9F_NUPH_HHE_20210402T094800.npy
Processing file: 9F_NUPH_HHE_20210402T104800.npy
Processing file: 9F_NUPH_HHE_20210402T114800.npy
Processing file: 9F_NUPH_HHE_20210402T124800.npy
Processing file: 9F_NUPH_HHE_20210402T134800.npy
Processing file: 9F_NUPH_HHE_20210402T144800.npy
Processing file: 9F_NUPH_HHE_20210402T154800.npy
Processing file: 9F_NUPH_HHE_20210402T164800.npy
Processing file: 9F_NUPH_HHE_20210402T174800.npy
Processing file: 9F_

In [33]:
# import matplotlib.pyplot as plt

# # Ensure there are spectrograms to plot
# if all_spectrograms.size > 0:
#     length = len(all_spectrograms)
#     print(length)
#     plt.figure(figsize=(10, 6))
#     for i in range(length):  # Loop through all spectrograms
#         plt.plot(all_spectrograms[i].flatten())
#     plt.title(f'Spectrograms as Line Plot')
#     plt.xlabel('Frequency Bins')
#     plt.ylabel('Log Power')
#     plt.show()
# else:
#     print("No spectrograms found in all_spectrograms.")

In [ ]:
# display information on Input.npy
print(f"Shape of combined spectrogram array with batch and channel dimensions: {all_spectrograms.shape}")
print(f"Number of files processed: {all_spectrograms.shape[0]}")

# load Input.npy and display information for confirmation
input_data_file_t = np.load(input_filepath)
print(f"Shape of combined spectrogram array Input.npy: {input_data_file_t.shape}")
print(f"Number of files processed: {input_data_file_t.shape[0]}")

# the numbers should match

Shape of combined spectrogram array with batch and channel dimensions: (670, 96, 128)
Number of files processed: 670


Shape of combined spectrogram array Input.npy: (670, 96, 128)
Number of files processed: 670


In [ ]:
# display first five rows in the all_spectograms
print(all_spectrograms[:5])

[[[ 0.9959167   0.9957651   0.9979812  ...  0.99752784  0.9975257
    0.99611413]
  [ 0.99730253  0.9969144   0.99717975 ...  0.9965135   0.99527013
    0.9960344 ]
  [ 0.99795735  0.99608743  0.99521327 ...  0.9963627   0.998163
    0.99723434]
  ...
  [-0.1760853  -0.1664002  -0.2237978  ... -0.32913667 -0.20307213
   -0.25544393]
  [-0.23450834 -0.12724859 -0.07112229 ... -0.23759365 -0.21076834
   -0.10105902]
  [-0.18473375 -0.19355106 -0.15950304 ... -0.19581515 -0.24280685
   -0.442913  ]]

 [[ 0.9961362   0.9970714   0.99762106 ...  0.9962281   0.9980012
    0.99909747]
  [ 0.9966816   0.9967321   0.9981055  ...  0.998225    0.99650896
    0.99756396]
  [ 0.99823713  0.99679387  0.99692523 ...  0.995247    0.9970925
    0.99790716]
  ...
  [-0.3243543  -0.30717146 -0.2709608  ... -0.20855331 -0.25421125
   -0.38084644]
  [-0.205688   -0.17286474 -0.28731835 ... -0.28536463 -0.15231007
   -0.13362432]
  [-0.22046787 -0.26096773 -0.25464946 ... -0.17267913 -0.30362517
   -0.42714